# N-gram Language Modeling

## Data

In [2]:
corpus = [
    "오늘 날씨 어때",
    "오늘 날씨 어때",
    "오늘 날씨 어때",
    "오늘 축구 경기",
    "오늘 경기 결과",
    "오늘 경기 결과",
    "내일 날씨",
    "내일 축구 경기",
    "내일 축구 경기",
    "축구 일정"
]

## Tokenization

In [3]:
vocabs = set()
for sample in corpus:
    for token in sample.split():
        vocabs.add(token)

In [4]:
vocabs = list(vocabs)

In [5]:
vocabs

['내일', '축구', '오늘', '날씨', '일정', '어때', '결과', '경기']

In [6]:
idx_to_token = vocabs
token_to_idx = {token: idx for idx, token in enumerate(idx_to_token)}

In [7]:
token_to_idx

{'내일': 0, '축구': 1, '오늘': 2, '날씨': 3, '일정': 4, '어때': 5, '결과': 6, '경기': 7}

---
## Bi-gram Language Modeling

In [8]:
prev_token_to_next_token_cnt_table = {}

In [9]:
total_cnt = 0

for sample in corpus:
    tokens = sample.split()

    for prev_token, next_token in zip(tokens[:-1], tokens[1:]):
        if prev_token not in prev_token_to_next_token_cnt_table:
            prev_token_to_next_token_cnt_table[prev_token] = {}
        if next_token not in prev_token_to_next_token_cnt_table[prev_token]:
            prev_token_to_next_token_cnt_table[prev_token][next_token] = 0
        prev_token_to_next_token_cnt_table[prev_token][next_token] += 1
        total_cnt += 1

In [10]:
prev_token_to_next_token_cnt_table

{'오늘': {'날씨': 3, '축구': 1, '경기': 2},
 '날씨': {'어때': 3},
 '축구': {'경기': 3, '일정': 1},
 '경기': {'결과': 2},
 '내일': {'날씨': 1, '축구': 2}}

In [11]:
import pandas as pd

In [12]:
prev_token_to_next_token_table_df = pd.DataFrame(prev_token_to_next_token_cnt_table).transpose()
prev_token_to_next_token_table_df.fillna(0,inplace=True)

In [13]:
prev_token_to_next_token_table_df

,날씨,축구,경기,어때,일정,결과
오늘,3.0,1.0,2.0,0.0,0.0,0.0
날씨,0.0,0.0,0.0,3.0,0.0,0.0
축구,0.0,0.0,3.0,0.0,1.0,0.0
경기,0.0,0.0,0.0,0.0,0.0,2.0
내일,1.0,2.0,0.0,0.0,0.0,0.0


In [14]:
prev_token_to_next_token_table_df.loc['오늘', '경기']

2.0

---
## P(prev_token, next_token) Joint Prob Distribution

In [15]:
prev_token_to_next_token_join_prob_table_df = prev_token_to_next_token_table_df / total_cnt

In [16]:
sum(prev_token_to_next_token_join_prob_table_df.sum())

1.0

In [17]:
prev_token_to_next_token_join_prob_table_df

,날씨,축구,경기,어때,일정,결과
오늘,0.166667,0.055556,0.111111,0.000000,0.000000,0.000000
날씨,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000
축구,0.000000,0.000000,0.166667,0.000000,0.055556,0.000000
경기,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111
내일,0.055556,0.111111,0.000000,0.000000,0.000000,0.000000


---
## P(next_token|prev_token) Conditional Prob Distribution

In [18]:
marginal_prob = prev_token_to_next_token_join_prob_table_df.sum(axis=1)

In [19]:
marginal_prob 

오늘    0.333333
날씨    0.166667
축구    0.222222
경기    0.111111
내일    0.166667
dtype: float64

In [20]:
prev_token_to_next_token_cond_prob_table_df = prev_token_to_next_token_join_prob_table_df.copy()

In [21]:
for row_name, row in prev_token_to_next_token_join_prob_table_df.iterrows():
    for col_name, val in row.items():
        prev_token_to_next_token_cond_prob_table_df.loc[row_name, col_name] /= marginal_prob[row_name]

In [22]:
prev_token_to_next_token_cond_prob_table_df

,날씨,축구,경기,어때,일정,결과
오늘,0.500000,0.166667,0.333333,0.0,0.00,0.0
날씨,0.000000,0.000000,0.000000,1.0,0.00,0.0
축구,0.000000,0.000000,0.750000,0.0,0.25,0.0
경기,0.000000,0.000000,0.000000,0.0,0.00,1.0
내일,0.333333,0.666667,0.000000,0.0,0.00,0.0


In [23]:
prev_token_to_next_token_cond_prob_table_df.loc["오늘", "날씨"]

0.5

In [24]:
prev_token_to_next_token_cond_prob_table_df.loc["경기", "결과"]

1.0

---
## Bi-gram을 이용한 Naive Autocomplete

In [25]:
x = "오늘"

input_tokens = x.split()

if input_tokens:
    last_token = input_tokens[-1]

prev_token_to_next_token_cond_prob_table_df.loc[last_token]

날씨    0.500000
축구    0.166667
경기    0.333333
어때    0.000000
일정    0.000000
결과    0.000000
Name: 오늘, dtype: float64

In [26]:
top_k = 2
next_token_candidates = prev_token_to_next_token_cond_prob_table_df.loc[last_token].sort_values(ascending=False)[:top_k]

In [27]:
next_token_candidates = next_token_candidates.keys().tolist()
next_token_candidates

['날씨', '경기']

In [28]:
next_token_candidates

['날씨', '경기']

In [29]:
def suggest(query, depth=2, top_k=2):

    input_tokens = query.split()

    if input_tokens:
        last_token = input_tokens[-1]
    else:
        last_token = marginal_prob.sort_values(ascending=False).index[0]
    
    suggested_list = []
    
    def dfs(last_token, prev_token_list, depth):

        if depth <= 0:
            return

        next_token_candidates = prev_token_to_next_token_cond_prob_table_df.loc[last_token].sort_values(ascending=False)
        next_token_candidates = next_token_candidates[next_token_candidates > 0]
        
        last_token_list = next_token_candidates.keys().tolist()[:top_k]

        for last_token in last_token_list:
            new_suggest = prev_token_list + [last_token]
            suggested_list.append(new_suggest)
            
            if last_token in prev_token_to_next_token_cond_prob_table_df.index:
                dfs(last_token, prev_token_list=new_suggest, depth=depth-1)
            
    dfs(last_token, [last_token], depth)
    
    suggested_list = [ " ".join(tokens) for tokens in suggested_list]
    return suggested_list

In [30]:
suggest("축구", depth=2)

['축구 경기', '축구 경기 결과', '축구 일정']

In [31]:
suggest("날씨", depth=2)

['날씨 어때']

In [32]:
suggest("경기", depth=2)

['경기 결과']

In [33]:
suggest("", depth=2)

['오늘 날씨', '오늘 날씨 어때', '오늘 경기', '오늘 경기 결과']